<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/04_19_GraphCodeBERT_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install tokenizers
!pip install pandas
!pip install torch
!pip install tqdm
!pip install scikit-learn
!pip install easydict

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer
from tqdm.auto import tqdm
import logging
from torch.utils.data import DataLoader, SequentialSampler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import os
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaModel
import pickle
from easydict import EasyDict as edict
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification


In [5]:
# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
# 인자 설정 클래스
args = edict({
    'train_data_file': '/content/drive/MyDrive/final_train5.csv',
    'eval_data_file': '/content/drive/MyDrive/final_val2.csv',
    'test_data_file': '/content/drive/MyDrive/final_test.csv',
    'output_dir': '/content/drive/My Drive/output',
    'model_name_or_path': 'microsoft/graphcodebert-base',
    'tokenizer_name': 'microsoft/graphcodebert-base',
    'block_size': 512,  # 입력 코드의 최대 토큰 수
    'use_logit_adjustment': True,  # 클래스 불균형을 고려한 로짓 조정 사용
    'tau': 1.2,  # 로짓 조정 계수
    'model_type': 'bert',  # 모델 아키텍처 유형
    'code_length': 256,  # 코드 길이, 필요에 따라 조정
    'do_train': True,
    'do_eval': True,
    'do_test': True,
    'evaluate_during_training': True,  # 훈련 중 평가 실행
    'do_local_explanation': False,
    'reasoning_method': None,
    'train_batch_size': 8,  # 훈련 배치 크기 증가
    'eval_batch_size': 8,  # 평가 배치 크기 증가
    'gradient_accumulation_steps': 1,
    'learning_rate': 3e-5,  # 학습률, BERT 기반 모델에 적합하도록 조정
    'weight_decay': 0.01,  # 가중치 감소, 일반적인 BERT 학습 설정
    'adam_epsilon': 1e-8,
    'max_grad_norm': 1.0,
    'max_steps': -1,
    'warmup_steps': 500,  # 워밍업 스텝, 필요에 따라 조정
    'seed': 42,
    'epochs': 2,  # 에폭 수 조정
    'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'n_gpu': torch.cuda.device_count(),  # 사용 가능한 GPU 수
    'cwe_label_map': {
        "Safe": 0,  # vul이 0인 경우, 즉 취약점이 없는 경우
        "CWE-20": 1,
        "CWE-119": 2,
        "CWE-78": 3,
        "CWE-122": 4,
        "CWE-121": 5,
        "CWE-415": 6,
        "CWE-399": 7,
        "CWE-190": 8,
        "CWE-125": 9,
        "CWE-416": 10
    }
})




In [7]:
# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(
    args.model_name_or_path,
    num_labels=len(args.cwe_label_map)  # CWE 유형의 수
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# InputFeatures 클래스 정의
class InputFeatures:
    """데이터의 한 세트의 특성을 정의합니다."""
    def __init__(self, input_ids, attention_mask, cwe_type_label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.cwe_type_label = cwe_type_label


In [9]:
# 특성 변환 함수
def convert_examples_to_features(func, cwe_type_label, tokenizer, max_length):
    """코드 스니펫을 모델 입력에 적합한 특성으로 변환합니다."""
    encoding = tokenizer.encode_plus(
        text=func,
        add_special_tokens=True,
        max_length=max_length,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    return InputFeatures(
        input_ids=encoding['input_ids'].flatten(),
        attention_mask=encoding['attention_mask'].flatten(),
        cwe_type_label=cwe_type_label
    )

In [26]:
# TextDataset 클래스 정의
class TextDataset(Dataset):
    def __init__(self, tokenizer, args, cwe_label_map, file_type="train"):
        self.tokenizer = tokenizer
        self.cwe_label_map = args.cwe_label_map
        file_path = getattr(args, f"{file_type}_data_file")
        self.examples = []
        df = pd.read_csv(file_path)
        funcs = df["code"].tolist()
        cwe_type_labels = df["labels"].tolist()

        for i in tqdm(range(len(funcs))):
            cwe_type_label = self.cwe_label_map.get(cwe_type_labels[i], 0)
            features = convert_examples_to_features(funcs[i], cwe_type_label, tokenizer, args.block_size)
            self.examples.append(features)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, labels):
      return {
        'input_ids': self.examples[labels].input_ids.clone().detach(),
        'attention_mask': self.examples[labels].attention_mask.clone().detach(),
        'labels': torch.tensor(self.examples[labels].cwe_type_label)  # labels로 변경
    }



In [11]:
# 데이터셋과 데이터 로더 생성
train_dataset = TextDataset(tokenizer, args, args.cwe_label_map, file_type='train')
eval_dataset = TextDataset(tokenizer, args, args.cwe_label_map, file_type='eval')
test_dataset = TextDataset(tokenizer, args, args.cwe_label_map, file_type='test')

train_loader = DataLoader(train_dataset, batch_size=args.train_batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=args.eval_batch_size, shuffle=False)


  0%|          | 0/13415 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  0%|          | 0/5749 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length

In [12]:
def compute_adjustment(tau, args, cwe_label_map):
    """클래스 빈도에 따라 로짓 조정 값을 계산합니다.

    Args:
        tau (float): 조정 계수, 클래스 빈도의 지수화에 사용됩니다.
        args (Args class): 설정 정보를 담고 있는 클래스 인스턴스.
        cwe_label_map (dict): 각 CWE ID에 대한 [index, one_hot, frequency] 정보를 담은 사전.

    Returns:
        torch.Tensor: 조정된 로짓 값들이 담긴 텐서.
    """
     # 클래스 빈도를 가정하는 단순화된 접근, 모든 클래스의 빈도가 동일하다고 가정
    num_classes = len(cwe_label_map)
    label_freq_array = np.ones(num_classes) / num_classes  # 모든 클래스에 대해 균등 분포 가정

    # 레이블 빈도를 PyTorch 텐서로 변환하고 로짓 조정 계산
    label_freq_tensor = torch.tensor(label_freq_array, dtype=torch.float32, device=args.device)
    adjustments = torch.log(label_freq_tensor ** tau + 1e-12)

    return adjustments


In [13]:
import random

def set_seed(args):
    """
    모든 난수 생성기의 시드를 설정하여 실험의 재현성을 보장합니다.

    Args:
    - args (Args class): 시드와 GPU 사용 여부를 포함하는 설정 클래스 인스턴스.

    이 함수는 Python의 내장 난수 생성기, NumPy, 그리고 PyTorch에 대해 동일한 시드를 설정합니다.
    또한, 여러 GPU를 사용하는 경우 각 디바이스의 CUDA 난수 생성기에 대해서도 시드를 설정합니다.
    """
    random.seed(args.seed)  # Python 내장 난수 생성기의 시드 설정
    np.random.seed(args.seed)  # NumPy 난수 생성기의 시드 설정
#    torch.manual_seed(args.seed)  # PyTorch 난수 생성기의 시드 설정

#    # CUDA 난수 생성기의 시드 설정 (GPU 사용 시)
#    if args.n_gpu > 0:
#        torch.cuda.manual_seed_all(args.seed)

# 예시로 Args 클래스를 정의하고 seed와 n_gpu를 설정합니다.
#class Args:
#    seed = 42
#    n_gpu = 1  # 이 값을 Colab에서 사용할 GPU 수에 맞춰 설정하세요.

#args = Args()
#set_seed(args)  # 시드 설정 함수 호출


In [14]:
from tqdm.auto import tqdm

def train(args, train_dataset, model, tokenizer):
    # 로짓 조정 사용 여부 확인 및 초기화
    if args.use_logit_adjustment:
        logit_adjustment = compute_adjustment(args.tau, args, args.cwe_label_map)
    else:
        logit_adjustment = None

    # 최적화 도구 및 스케줄러 설정
    optimizer = AdamW(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=args.max_steps)

    model.to(args.device)
    model.train()

    best_loss = float('inf')  # 최적 손실 초기화

    # 훈련 루프
    for epoch in range(args.epochs):
        # tqdm을 사용하여 진행 막대를 추가
        train_loader = DataLoader(train_dataset, batch_size=args.train_batch_size, shuffle=True)
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

        total_loss = 0
        for step, batch in enumerate(progress_bar):
            batch = {k: v.to(args.device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            if args.n_gpu > 1:
                loss = loss.mean()  # multi-gpu 지원
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            total_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})

            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

        # 에폭 종료 후 평가
        if eval_dataset is not None:
            eval_result = evaluate(args, model, tokenizer, eval_dataset)
            eval_loss = eval_result['eval_total_loss']  # 평가 손실 가져오기
            print(f"Epoch {epoch + 1} Evaluation Loss: {eval_loss:.4f}")

            # 로스 감소 시 체크포인트 저장
      #      if eval_loss < best_loss:
      #          best_loss = eval_loss
      #          save_checkpoint(model, args.output_dir, 'best_model.pt')

        progress_bar.close()
    logger.info("Training complete.")


In [15]:
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
logger = logging.getLogger(__name__)


def evaluate(args, model, tokenizer, eval_dataset):
    # DataLoader 생성
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, num_workers=0)

    # 모델 평가 모드 설정
    model.eval()
    total_loss = 0
    total_examples = 0

    # 평가 진행 with tqdm
    progress_bar = tqdm(eval_dataloader, desc="Evaluation", leave=False)

    logger.info("***** Running evaluation *****")
    logger.info("Num examples = %d", len(eval_dataset))
    logger.info("Batch size = %d", args.eval_batch_size)
    all_predictions = []
    all_labels = []
    for batch in progress_bar:
        batch = {k: v.to(args.device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()
            total_examples += batch['input_ids'].size(0)
            progress_bar.set_postfix({'loss': loss.item()})

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

    # 평균 손실 계산
    avg_loss = total_loss / total_examples


    progress_bar.close()

    # F1 스코어 계산
    f1_macro = f1_score(all_labels, all_predictions, average='macro')
    f1_micro = f1_score(all_labels, all_predictions, average='micro')
    print(f"F1 Macro: {f1_macro}, F1 Micro: {f1_micro}")

    # 결과 로깅
    logger.info("***** Eval results *****")
    logger.info("Total loss = %.4f", avg_loss)
    logger.info("F1 Macro = %.4f", f1_macro)
    logger.info("F1 Micro = %.4f", f1_micro)

    # 결과 반환
    result = {
        "eval_total_loss": avg_loss,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro
    }
    return result


In [16]:
logger = logging.getLogger(__name__)

from sklearn.metrics import accuracy_score, f1_score

def test(args, model, tokenizer, test_dataset, best_threshold=0.5):
    # DataLoader 구성
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.eval_batch_size, num_workers=0)

    # 다중 GPU 설정
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # 모델 평가 모드 설정
    model.eval()
    logger.info("***** Running Test *****")
    logger.info("Num examples = %d", len(test_dataset))
    logger.info("Batch size = %d", args.eval_batch_size)

    # 예측 및 실제 레이블 저장용 리스트
    cwe_type_preds = []
    cwe_type_trues = []

    # 테스트 데이터에 대한 평가
    for batch in test_dataloader:
        batch = {k: v.to(args.device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            cwe_type_preds.extend(predictions.cpu().numpy())
            cwe_type_trues.extend(batch['labels'].cpu().numpy())

    # 정확도 및 F1 스코어 계산
    cwe_type_acc = accuracy_score(cwe_type_trues, cwe_type_preds)
    cwe_type_f1 = f1_score(cwe_type_trues, cwe_type_preds, average='macro')
    print(f"CWE Type F1 Score: {cwe_type_f1}")
    result_cwe = {
        "CWE Type Accuracy": cwe_type_acc,
        "CWE Type F1 Score": cwe_type_f1
    }

    # 결과 로깅
    logger.info("***** CWE Type Classification Test Results *****")
    logger.info(f"CWE Type Accuracy = {cwe_type_acc:.4f}")
    logger.info(f"CWE Type F1 Score = {cwe_type_f1:.4f}")

    return result_cwe

In [17]:
# 설정된 디바이스와 GPU 정보 출력
logger.info(f"Device: {args.device}, Number of GPUs: {args.n_gpu}")

# Random seed 설정
#set_seed(args)

# Load tokenizers and models
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
num_labels = len(args.cwe_label_map)  # 클래스 개수
model = AutoModelForSequenceClassification.from_pretrained(
    args.model_name_or_path,
    num_labels=num_labels  # 모델 출력 레이어 설정
)
model.to(args.device)

# 데이터셋 로드
train_dataset = TextDataset(tokenizer, args, args.cwe_label_map, file_type='train')
eval_dataset = TextDataset(tokenizer, args, args.cwe_label_map, file_type='eval')
test_dataset = TextDataset(tokenizer, args, args.cwe_label_map, file_type='test')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/13415 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/5749 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/480 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# 훈련과 평가 실행
if args.do_train:
    logger.info("Starting training...")
    train(args, train_dataset, model, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1:   0%|          | 0/1677 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/719 [00:00<?, ?it/s]

F1 Macro: 0.7188812410405415, F1 Micro: 0.7470864498173595
Epoch 1 Evaluation Loss: 0.0848


Epoch 2:   0%|          | 0/1677 [00:00<?, ?it/s]

In [ ]:
if args.do_eval:
    logger.info("Starting evaluation...")
    evaluate(args, model, tokenizer, eval_dataset, eval_when_training=False)


In [ ]:
if args.do_test:
    logger.info("Starting testing...")
    test_results = test(args, model, tokenizer, test_dataset, best_threshold=0.5)
